In [1]:
print("Hello")

Hello


In [1]:
!pip install transformers
!pip install nltk


In [2]:
!pip install sentencepiece

In [3]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu, SmoothingFunction



In [4]:
def summarize_text(input_text, model, tokenizer):
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(input_ids, max_length=150, num_beams=4, length_penalty=2.0, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [5]:
def calculate_rouge(reference, hypothesis):
    smoothing = SmoothingFunction().method4
    rouge_score = sentence_bleu([reference], hypothesis, smoothing_function=smoothing)
    return rouge_score


In [6]:
input_paragraph = """
Natural language processing (NLP) is a field of artificial intelligence that focuses on the interaction between computers and humans using natural language. It enables computers to understand, interpret, and generate human-like text. NLP involves several challenges, including natural language understanding, language generation, and language translation. Text summarization is a specific NLP task that involves reducing the length of a document while retaining its key information. There are two main approaches to text summarization: extractive and abstractive. Extractive methods select important sentences from the original text, while abstractive methods generate new sentences to form the summary.
"""

In [7]:
model_name = "google/pegasus-xsum"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

In [8]:
summary = summarize_text(input_paragraph, model, tokenizer)



In [9]:
print("Original Paragraph:\n", input_paragraph)
print("\nSummary:\n", summary)

Original Paragraph:
 
Natural language processing (NLP) is a field of artificial intelligence that focuses on the interaction between computers and humans using natural language. It enables computers to understand, interpret, and generate human-like text. NLP involves several challenges, including natural language understanding, language generation, and language translation. Text summarization is a specific NLP task that involves reducing the length of a document while retaining its key information. There are two main approaches to text summarization: extractive and abstractive. Extractive methods select important sentences from the original text, while abstractive methods generate new sentences to form the summary.


Summary:
 Text summarization is a specific NLP task that involves reducing the length of a document while retaining its key information. Text summarization is a specific NLP task that involves reducing the length of a document while retaining its key information.


In [10]:
reference_summary = """
Natural language processing (NLP) is a field of artificial intelligence that focuses on the interaction between computers and humans using natural language
"""

In [11]:
rouge_score = calculate_rouge(reference_summary, summary)
print("\nROUGE Score:", rouge_score)



ROUGE Score: 0.18378541224195163


In [13]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=3b210f07c725955b84566899d8ad4cbec37a65fa28c1af280b9f8fac8d044dd9
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [14]:
from rouge_score import rouge_scorer

In [15]:
def calculate_rouge_score(reference, summary):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, summary)
    return scores

In [17]:
rouge_scores = calculate_rouge_score(reference_summary, summary)

In [18]:
print("\nRouge Scores:")
print(rouge_scores)


Rouge Scores:
{'rouge1': Score(precision=0.15, recall=0.2727272727272727, fmeasure=0.19354838709677416), 'rouge2': Score(precision=0.02564102564102564, recall=0.047619047619047616, fmeasure=0.03333333333333333), 'rougeL': Score(precision=0.125, recall=0.22727272727272727, fmeasure=0.16129032258064516)}
